In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.tools.tools import add_constant
from statsmodels.stats.diagnostic import (
    het_breuschpagan,
    het_white,
    linear_rainbow,
    linear_reset,
)
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.regressionplots import (
    plot_partregress_grid,
    influence_plot,
    plot_leverage_resid2,
)
from scipy.stats import shapiro, jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
MASTER = Path("../data/processed/master_teams_2000_2019.csv")
TAB_OUT = Path("../data/processed/")
FIG_OUT = Path("../plots/final_version")
LATEX_OUT = Path("../latex/tables/final_version")
for p in [TAB_OUT, FIG_OUT, LATEX_OUT]:
    p.mkdir(parents=True, exist_ok=True)

In [3]:
predictors_all = ["RunDiff", "ERA", "HR"]
target = "W"

In [4]:
df = pd.read_csv(MASTER, parse_dates=["season_date"])
sub = df[[target] + predictors_all].dropna().copy()

In [5]:
X = add_constant(sub[predictors_all])
y = sub[target]
model = sm.OLS(y, X).fit()

print("\n=== SUMMARY DEL MODELO MÚLTIPLE ===")
print(model.summary())


=== SUMMARY DEL MODELO MÚLTIPLE ===
                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     1534.
Date:                Wed, 01 Oct 2025   Prob (F-statistic):          9.94e-280
Time:                        00:51:22   Log-Likelihood:                -1681.4
No. Observations:                 600   AIC:                             3371.
Df Residuals:                     596   BIC:                             3388.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         8

In [6]:
yhat = model.fittedvalues.values
resid = model.resid.values

In [7]:
rainbow_stat, rainbow_p = linear_rainbow(model)

In [8]:
reset_res = linear_reset(model, power=2, use_f=True)
reset_F = float(reset_res.fvalue)
reset_p = float(reset_res.pvalue)

In [9]:
dw = sms.durbin_watson(resid)

In [10]:
bp_LM, bp_LMp, bp_F, bp_Fp = het_breuschpagan(resid, model.model.exog)
w_LM, w_LMp, w_F, w_Fp = het_white(resid, model.model.exog)

In [19]:
sh_W, sh_p = shapiro(resid)
jb_out = jarque_bera(resid)
try:
    jb_stat = float(jb_out.statistic)
    jb_p = float(jb_out.pvalue)
except AttributeError:
    jb_stat, jb_p = map(float, jb_out)

In [20]:
vif_rows = []
for i, col in enumerate(["const"] + predictors_all):
    vif_val = variance_inflation_factor(X.values, i)
    vif_rows.append({"variable": col, "VIF": float(vif_val)})
vif_df = pd.DataFrame(vif_rows)

In [21]:
cond_num = np.linalg.cond(X.values)

In [22]:
tests = pd.DataFrame(
    [
        {
            "prueba": "Rainbow (linealidad)",
            "estadístico": rainbow_stat,
            "p_value": rainbow_p,
        },
        {"prueba": "RESET Ramsey (F)", "estadístico": reset_F, "p_value": reset_p},
        {"prueba": "Durbin–Watson", "estadístico": dw, "p_value": np.nan},
        {"prueba": "Breusch–Pagan (LM)", "estadístico": bp_LM, "p_value": bp_LMp},
        {"prueba": "Breusch–Pagan (F)", "estadístico": bp_F, "p_value": bp_Fp},
        {"prueba": "White (LM)", "estadístico": w_LM, "p_value": w_LMp},
        {"prueba": "White (F)", "estadístico": w_F, "p_value": w_Fp},
        {"prueba": "Shapiro–Wilk", "estadístico": sh_W, "p_value": sh_p},
        {"prueba": "Jarque–Bera", "estadístico": jb_stat, "p_value": jb_p},
        {"prueba": "Número de condición", "estadístico": cond_num, "p_value": np.nan},
    ]
)

In [23]:
tests.to_csv(
    TAB_OUT / "ols_multiple_assumption_tests.csv", index=False, float_format="%.6f"
)
vif_df.to_csv(TAB_OUT / "ols_multiple_vif.csv", index=False, float_format="%.6f")
print("\nGuardados CSV: ols_multiple_assumption_tests.csv y ols_multiple_vif.csv")


Guardados CSV: ols_multiple_assumption_tests.csv y ols_multiple_vif.csv


In [24]:
latex_tests = tests.rename(
    columns={"prueba": "Prueba", "estadístico": "Estadístico", "p_value": "$p$"}
).to_latex(
    index=False,
    float_format="%.4f",
    caption="Pruebas de supuestos del modelo múltiple",
    label="tab:ols_multiple_assumption_tests",
)
with open(LATEX_OUT / "ols_multiple_assumption_tests.tex", "w") as f:
    f.write(latex_tests)

latex_vif = vif_df.rename(columns={"variable": "Variable"}).to_latex(
    index=False,
    float_format="%.4f",
    caption="Factor de Inflación de la Varianza (VIF) por predictor",
    label="tab:ols_multiple_vif",
)
with open(LATEX_OUT / "ols_multiple_vif.tex", "w") as f:
    f.write(latex_vif)

print(
    "Exportadas tablas LaTeX: ols_multiple_assumption_tests.tex y ols_multiple_vif.tex"
)

Exportadas tablas LaTeX: ols_multiple_assumption_tests.tex y ols_multiple_vif.tex


In [25]:
fig = plot_partregress_grid(model)
fig.suptitle("Partial regression (added-variable) – W ~ RunDiff + ERA + HR", y=1.02)
fig.tight_layout()
fig.savefig(
    FIG_OUT / "ols_partial_regression_grid.png",
    dpi=150,
    bbox_inches="tight",
    transparent=True,
)
plt.close(fig)

fig, ax = plt.subplots(figsize=(6.0, 4.5))
ax.scatter(yhat, resid, alpha=0.6)
ax.axhline(0.0, lw=1.0, color="black")
ax.set_xlabel("Valores ajustados (ŷ)")
ax.set_ylabel("Residuales (y - ŷ)")
ax.set_title("Residuales vs valores ajustados")
fig.tight_layout()
fig.savefig(
    FIG_OUT / "ols_residuals_vs_fitted.png",
    dpi=150,
    bbox_inches="tight",
    transparent=True,
)
plt.close(fig)

fig = qqplot(resid, line="45", fit=True)
plt.title("QQ-plot de residuales")
plt.tight_layout()
plt.savefig(
    FIG_OUT / "ols_residuals_qq.png", dpi=150, bbox_inches="tight", transparent=True
)
plt.close()

fig, ax = plt.subplots(figsize=(6.0, 4.5))
plot_leverage_resid2(model, ax=ax)
ax.set_title("Leverage vs residuales estandarizados")
fig.tight_layout()
fig.savefig(
    FIG_OUT / "ols_leverage_resid2.png", dpi=150, bbox_inches="tight", transparent=True
)
plt.close(fig)

fig, ax = plt.subplots(figsize=(6.5, 5.0))
influence_plot(model, criterion="cooks", ax=ax, alpha=0.7)
ax.set_title("Influence plot (Cook's distance)")
fig.tight_layout()
fig.savefig(
    FIG_OUT / "ols_influence_cooks.png", dpi=150, bbox_inches="tight", transparent=True
)
plt.close(fig)

In [26]:
print("\n=== VALIDACIÓN – RESUMEN ===")
print(tests.to_string(index=False))
print("\nVIF:")
print(vif_df.to_string(index=False))
print(f"\nFiguras guardadas en: {FIG_OUT}")


=== VALIDACIÓN – RESUMEN ===
              prueba  estadístico  p_value
Rainbow (linealidad)     1.051045 0.333950
    RESET Ramsey (F)     3.763321 0.052861
       Durbin–Watson     1.788626      NaN
  Breusch–Pagan (LM)     4.060384 0.255017
   Breusch–Pagan (F)     1.353598 0.256071
          White (LM)     9.720160 0.373611
           White (F)     1.079506 0.375869
        Shapiro–Wilk     0.996431 0.200004
         Jarque–Bera     3.840162 0.146595
 Número de condición  1979.312085      NaN

VIF:
variable        VIF
   const 116.649873
 RunDiff   3.172741
     ERA   2.679042
      HR   1.775557

Figuras guardadas en: ../plots/final_version
